In [1]:
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff

import seaborn as sns
from collections import Counter

In [2]:
train_file = open('train.txt', 'rb')

In [3]:
with open('train.txt') as f:
 #   line = f.readline()
 #   s_line = line.strip().split('\t')
 arr = []
 #   for item in s_line:
 #       print(item)
 #   print(s_line)
 #   text = []
 for line in f:
    s_line = line.strip().split('\t')
    arr.append(s_line)
print(max(arr, key=len))
print(len(arr))
print(arr[169])
 #       #text.append(splitted_line)

['111211', '__label__INSULT', '__label__OBSCENITY', '__label__THREAT', 'выебать петуха']
148775
['240089', '__label__NORMAL', 'работали даже в выходные нужен был план']


In [4]:
print(arr.index(max(arr,key=len)))

79


In [5]:
df = pd.DataFrame(columns=['id', 'normal', 'insult', 'obscenity', 'threat', 'comment'])

In [6]:
df.head()

,id,normal,insult,obscenity,threat,comment


In [7]:
to_append = max(arr,key=len)

In [26]:
to_append.insert(1, 0)

In [41]:
labels_dict = {'normal': '__label__NORMAL', 'insult': '__label__INSULT', 'obscenity': '__label__OBSCENITY', 'threat': '__label__THREAT'}

In [48]:
labels_dict.get('normal')

'__label__NORMAL'

In [49]:
labels_dict.values()

dict_values(['__label__NORMAL', '__label__INSULT', '__label__OBSCENITY', '__label__THREAT'])

In [63]:
def check_label(l_dict, item): 
    for label in l_dict.values():
            if item is label:
                print(item)
                return 1
            else:
                print(label)
                return 0
        

#  for sub in to_append: 
#      for i in sub.values(): 
#          if item == i: 
#              return True
#  return False

In [66]:
check_label(labels_dict.get('insult'), to_append[2])

AttributeError: 'str' object has no attribute 'values'

In [64]:
for item in to_append:
    print(item)
    # print(check_label(labels_dict, item))

111211
0
__label__INSULT
__label__OBSCENITY
__label__THREAT
выебать петуха


In [40]:
for key, value in labels_dict(k, v): 
    # res.append(check_label(ele, to_append)) 
    print(value)

NameError: name 'k' is not defined

In [32]:
for i in to_append:
    if i.ends:
        print(i)

SyntaxError: invalid syntax (<ipython-input-32-7213e29bef0b>, line 2)

In [27]:
to_append

['111211',
 0,
 '__label__INSULT',
 '__label__OBSCENITY',
 '__label__THREAT',
 'выебать петуха']

In [15]:
t_series = pd.DataFrame(to_append).transpose()

In [22]:
t_series.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       1 non-null      object
 1   1       1 non-null      object
 2   2       1 non-null      object
 3   3       1 non-null      object
 4   4       1 non-null      object
dtypes: object(5)
memory usage: 168.0+ bytes


In [21]:
t_series[t_series.columns.is_object()]

0    111211
Name: 0, dtype: object

In [17]:
to_append

['111211',
 '__label__INSULT',
 '__label__OBSCENITY',
 '__label__THREAT',
 'выебать петуха']

In [ ]:
df.append()

In [104]:
gg = zip(df.columns, max(arr,key=len))

In [107]:
s_dict = {'id': }
for i in arr:
    for j in i:
        
        print(type(j))
    #print(i)

ейка!!!
68262
__label__NORMAL
убивать детей это не правильно надо как_то их перевоспитывать.они боли не знают!
74551
__label__INSULT
по сегодняшним законам самые актуальные задачи,в первом случае отобрать и сожрать,во втором дать по рылу и будет справедливо.а вообще придумавшему премию долбоёба с медалью.
33573
__label__NORMAL
какое красивое поле
110089
__label__NORMAL
да девонька, труд твой не для ленивых.дай бог тебе сил и здоровья, счастья и доброго, верного друга по жизни !!!.радуй людей
120293
__label__NORMAL
когда нажрутся ?
42993
__label__NORMAL
тараканов и муравьев веками выводят, они живее всех живых(fr)
101257
__label__NORMAL
например мы с тобой
210646
__label__NORMAL
твои речи тебе на плечи, твои мысли на тебе повисли, что желаешь другим пусть сбудется тебе, а что желаешь себе отдай другим........
68936
__label__NORMAL
к сожалению такое положение не сегодняшнее. в1986 году и мне как папе пришлось спать на стуле рядом с больным ребенком. спасибо что разрешили.
61748
__label__

In [88]:
for i in arr:
    s = i.split('\t')
    print(s)
# bb = zip(*arr)

AttributeError: 'list' object has no attribute 'split'

In [85]:
bb[0]

TypeError: 'zip' object is not subscriptable

In [51]:
import itertools
# pd.DataFrame(list(map(dict, arr)))

tmp_df = pd.DataFrame((_ for _ in itertools.zip_longest(*arr))).T




In [52]:
tmp_df.head()

,0,1,2,3,4
0,41127,__label__THREAT,дворника надо тоже уничтожить!,None,None
1,6812,__label__NORMAL,"моя старшая неделю шипела, не принимала подкид...",None,None
2,6256,__label__NORMAL,полностью с вами согласна!,None,None
3,189636,__label__NORMAL,"хоть ногу вверх, ничего не изменится",None,None
4,99053,__label__NORMAL,а что значит - левого ребенка?,None,None


In [68]:
tmp_df.iloc[:, 2:]

,2,3,4
0,дворника надо тоже уничтожить!,None,None
1,"моя старшая неделю шипела, не принимала подкид...",None,None
2,полностью с вами согласна!,None,None
3,"хоть ногу вверх, ничего не изменится",None,None
4,а что значит - левого ребенка?,None,None
...,...,...,...
148770,"а вы думаете что полиция об этом не знает, про...",None,None
148771,"ну а что вы тут возмущаетесь , все ведь точно ...",None,None
148772,затерли до дыр данный ролик сколько лет ему уж...,None,None
148773,если проследят за пидором то накажут,None,None


In [72]:
tmp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148775 entries, 0 to 148774
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   0       148775 non-null  object
 1   1       148775 non-null  object
 2   2       148775 non-null  object
 3   3       4988 non-null    object
 4   4       111 non-null     object
dtypes: object(5)
memory usage: 5.7+ MB


In [81]:
tmp_df[pd.notnull(tmp_df.iloc[:, 3:])]

,0,1,2,3,4
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
148770,NaN,NaN,NaN,NaN,NaN
148771,NaN,NaN,NaN,NaN,NaN
148772,NaN,NaN,NaN,NaN,NaN
148773,NaN,NaN,NaN,NaN,NaN


In [14]:
text

м мен маст уша кун мен маст эдим мбекф я был нереально пьян а в тот день\n'],
 ['94538\t__label__NORMAL\tпрелесть 😂😂😂\n'],
 ['166117\t__label__NORMAL\tнормально молодец\n'],
 ['150103\t__label__NORMAL\t я за воробьева а. т. - с уважением, дударева ирина!\n'],
 ['202860\t__label__NORMAL\tсказочный до...\n'],
 ['185476\t__label__INSULT\tпидораас картофельный\n'],
 ['129473\t__label__NORMAL\tобратится к мастеру по ул заводская номера гезнаю его тел но он очень хорошо ремантирует и у него много деталей от машинок разных он в каневской много заказов делает просто я живуна этой улице и знаю дорого он не берет не пожалеете по моему заводская1 самый крайний домпо правой стороне\n'],
 ['84625\t__label__NORMAL\tкакой красавчик!!!🐶🐶\n'],
 ['8355\t__label__INSULT\t__label__OBSCENITY\tмамку я твою ипал срашка говно путин вор есвобода риss гиот америка это круто оппозиция россии выглядит примерно так vарlака сом\n'],
 ['140390\t__label__INSULT\tты пидор, вот это ты ушлёпок запомни точно!\n'],
 ['147

In [ ]:
all_data_df = pd.read_csv('data.txt', sep='\t', header=None)
all_data_df.columns = ['id', 'comment']

In [ ]:
train_df = pd.read_csv('train.txt', sep='\t', header=None)

In [ ]:
test_id_df = pd.read_csv('test.txt', sep='\t', header=None, names={'id'})

In [ ]:
train_df.head()

In [ ]:
print(all_data_df['id'].dtype)

In [ ]:
tmp_df = all_data_df.copy()

In [ ]:
def rename_cols(df_input):
    df_output = df_input.copy()
    for column in df_output.columns:
        print(df_output.column)
 #       if column.dtype == 'object' | column.dtype == 'str':
 #           column.rename('comment')
 #       elif column.dtype == 'int64':
 #           column.rename('id')
 #   return df_output"

In [ ]:
"# all_data_df = all_data_df.pipe(rename_cols)\n",
    "\n",
    "rename_cols(tmp_df)"

In [ ]:
all_data_df.head()

In [ ]:
all_data_df.rename(columns={'comment': 'id', 'id': 'comment'}, inplace=True)

In [ ]:
all_data_df['id'] = all_data_df['id'].astype('int64')

In [ ]:
test_df = pd.merge(all_data_df, test_id_df, on='id')

In [ ]:
print(all_data_df.shape, test_id_df.shape, train_df.shape, test_df.shape)

In [ ]:
len(train_df['label'].value_counts())

In [ ]:
temp = train_df.groupby('label').count()['comment'].reset_index().sort_values(by='comment',ascending=False)\n",
    "temp.style.background_gradient(cmap='Oranges')

In [ ]:
plt.figure(figsize=(12,6))\n",
    "sns.countplot(x='label',data=train_df)"

In [ ]:
fig = go.Figure(go.Funnelarea(\n",
    "    text = temp['label'],\n",
    "    values = temp['comment'],\n",
    "    title = {\"position\": \"top center\", \"text\": \"Funnel-Chart of Sentiment Distribution\"}\n",
    "    ))\n",
    "fig.show()

In [ ]:
from sklearn import preprocessing\n",
    "lb = preprocessing.LabelBinarizer()\n",
    "lb.fit(temp['label'])\n",
    "# LabelBinarizer()\n",
    "lb.classes_

In [ ]:
lb.transform(temp['label'])

In [ ]:
le = preprocessing.LabelEncoder()\n",
    "le.fit(temp['label'])\n",
    "le.classes_

In [ ]:
le_labels = le.transform(temp['label'])

In [ ]:
list(le.inverse_transform(le_labels))

In [ ]:
train_df['label_enc'] = le.transform(train_df['label'])

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv = CountVectorizer()

In [ ]:
m = cv.fit_transform(train_df['comment'])

In [ ]:
m

In [ ]:
"#del train_df['comment']\n",
    "del train_df['label']"

In [ ]:
train_df['label_enc'].hist(bins=len(set(train_df['label_enc'])))

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree\n",
    "from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier\n",
    "from sklearn.model_selection import train_test_split"

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(m, list(train_df['label_enc']))

In [ ]:
dtc = DecisionTreeClassifier(max_depth=10)

In [ ]:
rfc = RandomForestClassifier(n_jobs=-1)

In [ ]:
rfc.fit(X_train, y_train)

In [ ]:
dtc.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, classification_report

In [ ]:
preds = dtc.predict(X_test)

In [ ]:
print('Accuracy: ', float(accuracy_score(y_test, preds)*100), '%')\n",
    "print('Precision: ', precision_score(y_test, preds, average='macro'))\n",
    "print('Classification report: \\n', classification_report(y_test, preds))

In [ ]:
def calculate_metrics(self, submit):\n",
    "    df = self.answers.join(submit, on='id')\n",
    "\n",
    "    y_true = []\n",
    "    y_scores = []\n",
    "    for idx, row in df.iterrows():\n",
    "        answer_labels = row['labels'].split(',')\n",
    "        y_true.append([1 if label in answer_labels else 0 for label in LABELS])\n",
    "        y_scores.append([row['normal'], row['insult'], row['obscenity'], row['threat']])

In [ ]:
preds_r = rfc.predict(X_test)

In [ ]:
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 30,20
plot_tree(dtc, filled=True)